In [ ]:
# prompt: read excel file and show columns in file on a list

import pandas as pd

# Replace 'your_excel_file.xlsx' with the actual path to your Excel file
excel_file = '/content/data_dummy_perpustakaan.xlsx'

try:
    df = pd.read_excel(excel_file)
    columns = df.columns.tolist()
    print(columns)
except FileNotFoundError:
    print(f"Error: File not found at '{excel_file}'")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import pandas as pd
import numpy as np

# Membaca data dari file Excel
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Transformasi data ke dalam bentuk logaritma (menggunakan log natural)
df_log = df.apply(lambda x: np.log1p(x))  # log1p digunakan untuk menghindari log(0)

# Menampilkan DataFrame hasil transformasi
print(df_log)

# Jika ingin menyimpan hasil transformasi ke file CSV
df_log.to_csv("data_perpustakaan_log.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Membaca data dari file Excel
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Transformasi data ke dalam bentuk logaritma (menggunakan log natural)
df_log = df.apply(lambda x: np.log1p(x))  # log1p digunakan untuk menghindari log(0)

# Uji normalitas menggunakan Shapiro-Wilk
normality_results = {}
for col in df_log.columns:
    stat, p = stats.shapiro(df_log[col])
    normality_results[col] = {'Statistic': stat, 'p-value': p}

# Menampilkan hasil uji normalitas
for col, res in normality_results.items():
    print(f"{col}: Statistic={res['Statistic']}, p-value={res['p-value']}")

# Visualisasi distribusi data
plt.figure(figsize=(12, 8))
for i, col in enumerate(df_log.columns, 1):
    plt.subplot(4, 4, i)
    sns.histplot(df_log[col], kde=True)
    plt.title(col)
plt.tight_layout()
plt.show()

# Jika ingin menyimpan hasil transformasi ke file CSV
df_log.to_csv("data_perpustakaan_log.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer

# Membaca data dari file Excel
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Pastikan semua data bertipe numerik
df = df.apply(pd.to_numeric, errors='coerce')

# Pisahkan kolom dengan nilai positif (untuk Box-Cox) dan campuran (untuk Yeo-Johnson)
positive_cols = [col for col in df.columns if (df[col] > 0).all()]
mixed_cols = [col for col in df.columns if col not in positive_cols]

# Terapkan Box-Cox untuk kolom dengan nilai positif jika ada
if positive_cols:
    pt_boxcox = PowerTransformer(method='box-cox')
    df_boxcox = pd.DataFrame(pt_boxcox.fit_transform(df[positive_cols]), columns=positive_cols)
else:
    df_boxcox = pd.DataFrame()

# Terapkan Yeo-Johnson untuk kolom dengan nilai nol atau negatif jika ada
if mixed_cols:
    pt_yeojohnson = PowerTransformer(method='yeo-johnson')
    df_yeojohnson = pd.DataFrame(pt_yeojohnson.fit_transform(df[mixed_cols]), columns=mixed_cols)
else:
    df_yeojohnson = pd.DataFrame()

# Gabungkan hasil transformasi
df_transformed = pd.concat([df_boxcox, df_yeojohnson], axis=1)

# Uji normalitas menggunakan Shapiro-Wilk
normality_results = {}
for col in df_transformed.columns:
    stat, p = stats.shapiro(df_transformed[col])
    normality_results[col] = {'Statistic': stat, 'p-value': p}

# Menampilkan hasil uji normalitas
for col, res in normality_results.items():
    print(f"{col}: Statistic={res['Statistic']}, p-value={res['p-value']}")

# Visualisasi distribusi data
plt.figure(figsize=(12, 8))
for i, col in enumerate(df_transformed.columns, 1):
    plt.subplot(4, 4, i)
    sns.histplot(df_transformed[col], kde=True)
    plt.title(col)
plt.tight_layout()
plt.show()

# Jika ingin menyimpan hasil transformasi ke file CSV
df_transformed.to_csv("data_perpustakaan_transformed.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Membaca data dari file Excel
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Menentukan epsilon untuk menghindari pembagian dengan nol
epsilon = 1e-6

# Terapkan Reciprocal Transformation (1/X)
df_reciprocal = df.copy()
for col in df.columns:
    df_reciprocal[col] = 1 / (df[col] + epsilon)

# Uji normalitas menggunakan Shapiro-Wilk setelah transformasi
normality_results_reciprocal = {}
for col in df_reciprocal.columns:
    stat, p = stats.shapiro(df_reciprocal[col])
    normality_results_reciprocal[col] = {'Statistic': stat, 'p-value': p}

# Menampilkan hasil uji normalitas
for col, res in normality_results_reciprocal.items():
    print(f"{col}: Statistic={res['Statistic']}, p-value={res['p-value']}")

# Visualisasi distribusi data setelah Reciprocal Transformation
plt.figure(figsize=(12, 8))
for i, col in enumerate(df_reciprocal.columns, 1):
    plt.subplot(4, 4, i)
    sns.histplot(df_reciprocal[col], kde=True)
    plt.title(col)
plt.tight_layout()
plt.show()

# Jika ingin menyimpan hasil transformasi ke file CSV
df_reciprocal.to_csv("data_perpustakaan_reciprocal.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Membaca data
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Normalisasi Min-Max Scaling (agar semua variabel memiliki rentang 0-1)
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Tentukan bobot untuk setiap variabel (misalnya bobot yang diberikan berdasarkan kepentingan)
weights = {
    'koleksi_tercetak': 0.15,
    'koleksi_digital': 0.15,
    'tercetak_dimanfaatkan': 0.10,
    'digital_dimanfaatkan': 0.10,
    'penambahan_tercetak': 0.10,
    'penambahan_digital': 0.05,
    'tenaga_pustakawan': 0.10,
    'pustakawan_teknis': 0.05,
    'pelatihan_pustakawan': 0.05,
    'peserta_literasi': 0.05,
    'pengguna_perpustakaan': 0.05,
    'pengguna_TIK': 0.05,
    'kegiatan_literasi': 0.05
}

# Pastikan bobot total = 1
total_weight = sum(weights.values())

# Hitung Indeks sebagai Weighted Average Index
df_scaled['Indeks_Perpustakaan'] = sum(df_scaled[col] * weights[col] for col in weights)

# Simpan hasil ke file CSV
df_scaled.to_csv("indeks_perpustakaan.csv", index=False)

# Tampilkan 5 data pertama
print(df_scaled[['Indeks_Perpustakaan']].head())


In [ ]:
print(df_scaled['Indeks_Perpustakaan'].describe())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.histplot(df_scaled['Indeks_Perpustakaan'], bins=10, kde=True)
plt.xlabel('Indeks Perpustakaan')
plt.ylabel('Frekuensi')
plt.title('Distribusi Indeks Perpustakaan')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Membaca data
df = pd.read_excel("/content/data_dummy_perpustakaan.xlsx")

# Pisahkan kolom numerik dan non-numerik
numerical_cols = df.select_dtypes(include=['number']).columns  # Hanya kolom numerik
categorical_cols = df.select_dtypes(exclude=['number']).columns  # Kolom non-numerik (teks)

# Normalisasi Min-Max Scaling (hanya untuk kolom numerik)
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=numerical_cols)

# Kembalikan kolom non-numerik (misalnya Nama Kota/Provinsi)
df_scaled[categorical_cols] = df[categorical_cols]

# Tentukan bobot untuk setiap variabel numerik
weights = {
    'koleksi_tercetak': 0.15,
    'koleksi_digital': 0.15,
    'tercetak_dimanfaatkan': 0.10,
    'digital_dimanfaatkan': 0.10,
    'penambahan_tercetak': 0.10,
    'penambahan_digital': 0.05,
    'tenaga_pustakawan': 0.10,
    'pustakawan_teknis': 0.05,
    'pelatihan_pustakawan': 0.05,
    'peserta_literasi': 0.05,
    'pengguna_perpustakaan': 0.05,
    'pengguna_TIK': 0.05,
    'kegiatan_literasi': 0.05
}

# Pastikan semua bobot dijumlahkan menjadi 1
total_weight = sum(weights.values())

# Hitung Indeks sebagai Weighted Average Index
df_scaled['Indeks_Perpustakaan'] = sum(df_scaled[col] * weights[col] for col in weights)

# Simpan hasil ke file CSV
df_scaled.to_csv("indeks_perpustakaan.csv", index=False)

# Tampilkan 5 data pertama (termasuk kolom wilayah agar lebih jelas)
print(df_scaled[['Indeks_Perpustakaan'] + list(categorical_cols)].head())


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

# Ambil data indeks perpustakaan
data = df_scaled['Indeks_Perpustakaan']

# Hitung mean dan standar deviasi
mean = np.mean(data)
std_dev = np.std(data)

# Buat rentang nilai untuk distribusi normal
x = np.linspace(min(data), max(data), 100)
y = norm.pdf(x, mean, std_dev)

# Plot histogram & kurva Bell
plt.figure(figsize=(8, 5))
sns.histplot(data, bins=20, kde=True, color='blue', stat='density', alpha=0.3, label="Histogram")
plt.plot(x, y, color='red', linewidth=2, label="Kurva Bell (Normal)")

# Tambahkan garis Mean dan SD
plt.axvline(mean, color='green', linestyle='dashed', linewidth=2, label=f"Mean = {mean:.2f}")
plt.axvline(mean - std_dev, color='orange', linestyle='dashed', linewidth=2, label="-1 SD")
plt.axvline(mean + std_dev, color='orange', linestyle='dashed', linewidth=2, label="+1 SD")

# Tambahkan label dan judul
plt.title("Distribusi Indeks Perpustakaan (Kurva Bell)")
plt.xlabel("Indeks Perpustakaan")
plt.ylabel("Density")
plt.legend()
plt.grid(True)

# Tampilkan plot
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

# Ambil data indeks perpustakaan dan kabupaten/kota
data = df_scaled[['KABUPATEN/KOTA', 'Indeks_Perpustakaan']]

# Hitung mean dan standar deviasi
mean = np.mean(data['Indeks_Perpustakaan'])
std_dev = np.std(data['Indeks_Perpustakaan'])

# Buat rentang nilai untuk distribusi normal
x = np.linspace(min(data['Indeks_Perpustakaan']), max(data['Indeks_Perpustakaan']), 100)
y = norm.pdf(x, mean, std_dev)

# Plot histogram & kurva Bell
plt.figure(figsize=(10, 6))
sns.histplot(data['Indeks_Perpustakaan'], bins=20, kde=True, color='blue', stat='density', alpha=0.3, label="Histogram")
plt.plot(x, y, color='red', linewidth=2, label="Kurva Bell (Normal)")

# Tambahkan garis Mean dan SD
plt.axvline(mean, color='green', linestyle='dashed', linewidth=2, label=f"Mean = {mean:.2f}")
plt.axvline(mean - std_dev, color='orange', linestyle='dashed', linewidth=2, label="-1 SD")
plt.axvline(mean + std_dev, color='orange', linestyle='dashed', linewidth=2, label="+1 SD")

# Tambahkan label nama kabupaten/kota di titik data
for i, row in data.iterrows():
    plt.text(row['Indeks_Perpustakaan'], 0.02, row['KABUPATEN/KOTA'], fontsize=8, rotation=45, ha='right')

# Tambahkan label dan judul
plt.title("Distribusi Indeks Perpustakaan (Kurva Bell) dengan Label Kabupaten/Kota")
plt.xlabel("Indeks Perpustakaan")
plt.ylabel("Density")
plt.legend()
plt.grid(True)

# Tampilkan plot
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

# Ambil data indeks perpustakaan dan kabupaten/kota
data = df_scaled[['KABUPATEN/KOTA', 'Indeks_Perpustakaan']]

# Hitung mean dan standar deviasi
mean = np.mean(data['Indeks_Perpustakaan'])
std_dev = np.std(data['Indeks_Perpustakaan'])

# Buat rentang nilai untuk distribusi normal
x = np.linspace(min(data['Indeks_Perpustakaan']), max(data['Indeks_Perpustakaan']), 100)
y = norm.pdf(x, mean, std_dev)

# Plot histogram & kurva Bell
plt.figure(figsize=(10, 6))
sns.histplot(data['Indeks_Perpustakaan'], bins=20, kde=True, color='blue', stat='density', alpha=0.3, label="Histogram")
plt.plot(x, y, color='red', linewidth=2, label="Kurva Bell (Normal)")

# Tambahkan garis Mean dan SD
plt.axvline(mean, color='green', linestyle='dashed', linewidth=2, label=f"Mean = {mean:.2f}")
plt.axvline(mean - std_dev, color='orange', linestyle='dashed', linewidth=2, label="-1 SD")
plt.axvline(mean + std_dev, color='orange', linestyle='dashed', linewidth=2, label="+1 SD")

# Tambahkan label nama kabupaten/kota di titik data
for i, row in data.iterrows():
    plt.text(row['Indeks_Perpustakaan'], 0.02, row['KABUPATEN/KOTA'], fontsize=8, rotation=45, ha='right')

# Tambahkan label dan judul
plt.title("Distribusi Indeks Perpustakaan (Kurva Bell) dengan Label Kabupaten/Kota")
plt.xlabel("Indeks Perpustakaan")
plt.ylabel("Density")
plt.legend()
plt.grid(True)

# Tampilkan plot
plt.show()

# Hitung mean dan standar deviasi
mean = np.mean(data['Indeks_Perpustakaan'])
std_dev = np.std(data['Indeks_Perpustakaan'])

# Buat rentang nilai distribusi normal
x = np.linspace(data['Indeks_Perpustakaan'].min(), data['Indeks_Perpustakaan'].max(), 100)
y = norm.pdf(x, mean, std_dev)

# Buat grafik interaktif
fig = go.Figure()

# Histogram dengan transparansi
fig.add_trace(go.Histogram(
    x=data['Indeks_Perpustakaan'],
    histnorm='probability density',
    name="Histogram",
    opacity=0.4,
    marker=dict(color='blue')
))

# Tambahkan Kurva Bell
fig.add_trace(go.Scatter(
    x=x, y=y, mode='lines', name='Kurva Bell (Normal)',
    line=dict(color='red', width=2)
))

# Tambahkan Mean dan Standar Deviasi sebagai garis vertikal
fig.add_trace(go.Scatter(
    x=[mean], y=[max(y)], mode='markers+text',
    name="Mean", marker=dict(color='green', size=10),
    text=[f"Mean: {mean:.2f}"], textposition="top center"
))

fig.add_trace(go.Scatter(
    x=[mean - std_dev, mean + std_dev], y=[max(y)/2, max(y)/2],
    mode='markers+text', name="±1 SD",
    marker=dict(color='orange', size=10),
    text=["-1 SD", "+1 SD"], textposition="top center"
))

# Tambahkan titik data untuk Kabupaten/Kota dengan hover info
fig.add_trace(go.Scatter(
    x=data['Indeks_Perpustakaan'],
    y=[0] * len(data),
    mode='markers+text',
    text=data['KABUPATEN/KOTA'],
    name="Kabupaten/Kota",
    textposition="bottom center",
    marker=dict(size=8, color='blue')
))

# Layout interaktif
fig.update_layout(
    title="Distribusi Indeks Perpustakaan (Interaktif)",
    xaxis_title="Indeks Perpustakaan",
    yaxis_title="Density",
    hovermode="x unified"
)

# Tampilkan grafik
fig.show()
